In [411]:
import pulp
import pandas as pd
import numpy as np

In [423]:
#Reading the data; Due to presence of some special characters, encoding argument had to be given
football_players_full = pd.read_csv("LP.csv", index_col = "Name", encoding = 'ISO-8859-1')

In [424]:
#subsetting the top 50 rows - we will selecting our team from this pool
football_players_subset = football_players_full.iloc[0:49,  ]

In [425]:
#subsetting the required columns - these columns are the attributes which go into our constraints, DV
football_players = football_players_subset.iloc[:,[0,2,7,8,11,15,17,19,20,21,22,23,24,25,26,28,29,30,32,33,35,36,38,41]]

In [426]:
football_players.head(2)

,Age,Overall,Vvalue,Wwage,Acceleration,Ball control,Crossing,Dribbling,Finishing,Free kick accuracy,...,GK reflexes,Interceptions,Jumping,Long passing,Marking,Penalties,Reactions,Short passing,Sliding tackle,Standing tackle
Name,,,,,,,,,,,,,,,,,,,,,
Cristiano Ronaldo,32,94,95500000,565000,89,93,85,91,94,76,...,11,29,95,77,22,85,96,83,23,31
L. Messi,30,93,105000000,565000,92,95,77,97,95,90,...,8,22,68,87,13,74,95,88,26,28


In [427]:
#creating the players as the decision variable. They are chosen as binary - so that later they can be modelled 
#for 1 as selected and 0 as not selected
player_select = pulp.LpVariable.dicts("player_select", ((Name) for Name in football_players.index), cat='Binary')

In [428]:
# the objective function is to maximize the player ratings - in other words best possible team as determined by the
#"overall" values in the data under the constraints

model = pulp.LpProblem("Player selection problem", pulp.LpMaximize)
model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Overall"] for Name in football_players.index])

#Providing the cost constraint
model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Vvalue"] for Name in football_players.index])<=643390000

#creating the team size constraint
team_size = 0
for Name in football_players.index:
    team_size += player_select[Name]
model += team_size ==11

In [429]:
#converting values in the skill attributes to 0 and 1. 0 if they are below the threshold defined in the constraints
#(proposal document) and 1 if they are above the threshold value
#this conversion will help in creating the constraints where we need a subset of players who are strong in a 
#particular skillset

football_players["GK diving"] = np.where(football_players["GK diving"] > 85,1,0)
football_players["GK handling"] = np.where(football_players["GK handling"] > 85,1,0)
football_players["GK kicking"] = np.where(football_players["GK kicking"] > 85,1,0)
football_players["GK positioning"] = np.where(football_players["GK positioning"] > 85,1,0)
football_players["GK reflexes"] = np.where(football_players["GK reflexes"] > 85,1,0)

football_players["Interceptions"] = np.where(football_players["Interceptions"] > 80,1,0)
football_players["Marking"] = np.where(football_players["Marking"] > 80,1,0)

football_players["Sliding tackle"] = np.where(football_players["Sliding tackle"] > 75,1,0)
football_players["Standing tackle"] = np.where(football_players["Standing tackle"] > 83,1,0)

football_players["Short passing"] = np.where(football_players["Short passing"] > 80,1,0)
football_players["Long passing"] = np.where(football_players["Long passing"] > 80,1,0)

football_players["Reactions"] = np.where(football_players["Reactions"] > 85,1,0)
football_players["Jumping"] = np.where(football_players["Jumping"] > 75,1,0)

football_players["Finishing"] = np.where(football_players["Finishing"] > 90,1,0)
football_players["Acceleration"] = np.where(football_players["Acceleration"] > 80,1,0)

football_players["Ball control"] = np.where(football_players["Ball control"] > 80,1,0)
football_players["Crossing"] = np.where(football_players["Crossing"] > 80,1,0)

football_players["Dribbling"] = np.where(football_players["Dribbling"] > 80,1,0)
football_players["Penalties"] = np.where(football_players["Penalties"] > 75,1,0)

football_players["Free kick accuracy"] = np.where(football_players["Free kick accuracy"] > 87,1,0)





/Users/anon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/anon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/anon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/panda

In [430]:
#creating the constraints on the above skill attributes

model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"GK diving"] for Name in football_players.index])==1
model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"GK handling"] for Name in football_players.index])==1

model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"GK kicking"] for Name in football_players.index])==1
model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"GK positioning"] for Name in football_players.index])==1

model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"GK reflexes"] for Name in football_players.index])==1

model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Interceptions"] for Name in football_players.index])>=4
model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Marking"] for Name in football_players.index])>=4


model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Sliding tackle"] for Name in football_players.index])>=2
model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Standing tackle"] for Name in football_players.index])>=4



model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Short passing"] for Name in football_players.index])>=6

model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Long passing"] for Name in football_players.index])>=4

model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Reactions"] for Name in football_players.index])>=3

model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Jumping"] for Name in football_players.index])>=3

model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Finishing"] for Name in football_players.index])>=1

model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Acceleration"] for Name in football_players.index])>=3


model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Ball control"] for Name in football_players.index])>=4

model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Crossing"] for Name in football_players.index])>=2

model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Dribbling"] for Name in football_players.index])>=2


model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Penalties"] for Name in football_players.index])>=5

model += pulp.lpSum([player_select[Name] * football_players.loc[(Name),"Free kick accuracy"] for Name in football_players.index])>=1





In [431]:
#Solving the model
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [432]:
#looking at the output
output = []
for Name in player_select:
    var_output = {
        'Name': Name,
        
        'player_select': player_select[(Name)].varValue,
        
    }
    output.append(var_output)
output

[{'Name': 'Cristiano Ronaldo', 'player_select': 1.0},
 {'Name': 'L. Messi', 'player_select': 1.0},
 {'Name': 'Neymar', 'player_select': 0.0},
 {'Name': 'L. Suarez', 'player_select': 1.0},
 {'Name': 'M. Neuer', 'player_select': 1.0},
 {'Name': 'R. Lewandowski', 'player_select': 0.0},
 {'Name': 'De Gea', 'player_select': 0.0},
 {'Name': 'E. Hazard', 'player_select': 0.0},
 {'Name': 'T. Kroos', 'player_select': 0.0},
 {'Name': 'G. Higuaín', 'player_select': 0.0},
 {'Name': 'Sergio Ramos', 'player_select': 1.0},
 {'Name': 'K. De Bruyne', 'player_select': 0.0},
 {'Name': 'T. Courtois', 'player_select': 0.0},
 {'Name': 'A. Sánchez', 'player_select': 0.0},
 {'Name': 'L. Modri?', 'player_select': 0.0},
 {'Name': 'G. Bale', 'player_select': 0.0},
 {'Name': 'S. Agüero', 'player_select': 0.0},
 {'Name': 'G. Chiellini', 'player_select': 1.0},
 {'Name': 'G. Buffon', 'player_select': 0.0},
 {'Name': 'P. Dybala', 'player_select': 0.0},
 {'Name': 'J. Oblak', 'player_select': 0.0},
 {'Name': 'A. Griezm

In [422]:
58490000*11

643390000